In [21]:
from time import time
import logging
import pylab as pl
import numpy as np
import PIL

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report,confusion_matrix,f1_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [4]:
###############################################################################
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)

# for machine learning we use the data directly (as relative pixel
# position info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print "Total dataset size:"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_classes: %d" % n_classes

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [9]:
# Split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

###############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 150

print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
t0 = time()
pca = PCA(n_components=n_components, whiten=True).fit(X_train)
print "done in %0.3fs" % (time() - t0)

eigenfaces = pca.components_.reshape((n_components, h, w))

print "Projecting the input data on the eigenfaces orthonormal basis"
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print "done in %0.3fs" % (time() - t0)


Extracting the top 150 eigenfaces from 966 faces
done in 0.170s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.018s


# Explained Variance of Each pc

In [16]:
var_ratio = pca.explained_variance_ratio_
print('first component explained {0}'.format(round(var_ratio[0],3)))
print('second component explained {0}'.format(round(var_ratio[1],3)))

first component explained 0.193
second component explained 0.151


# F1 Score vs. No. of PCs Used

In [62]:

for k in [10, 15, 25, 50, 100, 250]:
    pca = PCA(n_components=k,whiten=True).fit(X_train)

    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
    # for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    
    y_pred = clf.predict(X_test_pca)
    
    print '{0} '.format(k) + classification_report(y_test, y_pred, target_names=target_names).split('\n')[2]

10      Ariel Sharon       0.10      0.15      0.12        13
15      Ariel Sharon       0.24      0.46      0.32        13
25      Ariel Sharon       0.60      0.69      0.64        13
50      Ariel Sharon       0.67      0.77      0.71        13
100      Ariel Sharon       0.64      0.69      0.67        13
250      Ariel Sharon       0.56      0.69      0.62        13
